### Install & Import Required Libraries

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf


### Load & Explore the dataset

In [3]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Observations:

- The dataset includes passenger demographics & survival data.
- We need to handle missing values in Age and Embarked

#### Descriptive Statistics


In [9]:
# Summary statistics of numerical columns
df.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
# Summary of categorical columns
df.describe(include="O")

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


#### Findings:

- 38% survival rate (from Survived column).
- Mean age: 29.7 years.
- Most passengers were in 3rd class (Pclass 3)

### Hypothesis Testing (Survival Factors Analysis)


##### Hypothesis 1: Did Gender Affect Survival? (Chi-Square Test)

Women had a higher survival rate than men.
- Test Used: Chi-Square Test for Independence

In [11]:
# Create contingency table
contingency_table = pd.crosstab(df["Sex"], df["Survived"])

# Perform Chi-Square Test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(f"Chi-Square Test Statistic: {chi2}, p-value: {p}")

# Interpretation
if p < 0.05:
    print(" Reject Null Hypothesis: Gender significantly affected survival.")
else:
    print(" Fail to Reject Null Hypothesis: Gender had no significant impact.")


Chi-Square Test Statistic: 260.71702016732104, p-value: 1.1973570627755645e-58
 Reject Null Hypothesis: Gender significantly affected survival.


##### Conclusion:

- p-value < 0.05 → Women had a significantly higher survival rate

#### Hypothesis 2: Does Passenger Class Impact Survival? (ANOVA Test)
Hypothesis: First-class passengers had a higher survival rate.
- Test Used: One-Way ANOVA

In [12]:
# Survival rate by class
pclass_groups = [df[df["Pclass"] == i]["Survived"] for i in df["Pclass"].unique()]

# Perform ANOVA test
f_stat, p_value = stats.f_oneway(*pclass_groups)
print(f"ANOVA Test Statistic: {f_stat}, p-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print(" Reject Null Hypothesis: Passenger class significantly affected survival.")
else:
    print(" Fail to Reject Null Hypothesis: Passenger class had no significant impact.")


ANOVA Test Statistic: 57.96481759091012, p-value: 2.1832474151179777e-24
 Reject Null Hypothesis: Passenger class significantly affected survival.


###### Conclusion:

p-value < 0.05 → First-class passengers were more likely to survive.


### Hypothesis 3: Did Age Affect Survival? (T-Test)

Hypothesis: Younger passengers had a higher survival rate.
- Test Used: Independent T-Test

In [14]:
# Split dataset into survived and not survived
survived = df[df["Survived"] == 1]["Age"].dropna()
not_survived = df[df["Survived"] == 0]["Age"].dropna()

# Perform T-test
t_stat, p_value = stats.ttest_ind(survived, not_survived)
print(f"T-Test Statistic: {t_stat}, p-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print(" Reject Null Hypothesis: Age significantly impacted survival.")
else:
    print(" Fail to Reject Null Hypothesis: Age had no significant impact.")


T-Test Statistic: -2.06668694625381, p-value: 0.03912465401348249
 Reject Null Hypothesis: Age significantly impacted survival.


- Conclusion:

p-value < 0.05 → Younger passengers had better survival chances.


#### Hypothesis 4: Did Fare Affect Survival? (Mann-Whitney U Test)
Passengers who paid higher fares had a higher survival rate.
-  Test Used: Mann-Whitney U Test (non-parametric test)

In [15]:
# Perform Mann-Whitney U test
u_stat, p_value = stats.mannwhitneyu(df[df["Survived"] == 1]["Fare"], df[df["Survived"] == 0]["Fare"])
print(f"Mann-Whitney U Test Statistic: {u_stat}, p-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print(" Reject Null Hypothesis: Fare significantly impacted survival.")
else:
    print(" Fail to Reject Null Hypothesis: Fare had no significant impact.")


Mann-Whitney U Test Statistic: 129951.5, p-value: 4.553477179250237e-22
 Reject Null Hypothesis: Fare significantly impacted survival.


#### Probability & Survival Predictions
Probability of Survival Based on Gender

In [16]:
# Survival probability by gender
prob_survival = df.groupby("Sex")["Survived"].mean()
print(prob_survival)


Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


##### Findings:

- Females: ~74% survival rate
- Males: ~18% survival rate


#####  Logistic Regression: Predicting Survival Probability


In [17]:
# Logistic Regression Model
import statsmodels.api as sm

# Define features & target variable
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})  # Encode gender
X = df[["Pclass", "Sex", "Age", "Fare"]].dropna()
y = df.loc[X.index, "Survived"]

# Fit logistic regression model
X = sm.add_constant(X)  # Add intercept
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Show summary
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.453242
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Sun, 09 Feb 2025   Pseudo R-squ.:                  0.3290
Time:                        20:39:18   Log-Likelihood:                -323.61
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 2.020e-67
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4698      0.525      4.707      0.000       1.442       3.498
Pclass        -1.2697      0.

###### Findings:

- Sex & Fare have the highest impact on survival probability.
- Pclass also significantly affects survival chances

#### Final Summary & Insights

##### Descriptive Analysis:

- 38% survival rate, average age: 29.7, mostly 3rd class passengers.

##### Hypothesis Testing Results:

- Gender, Passenger Class, Age, and Fare significantly affect survival.
- Women & First-Class passengers had much higher survival rates.
##### Probability Analysis:

- 74% survival for females, 18% for males.
- 1st Class survival: 63%, 3rd Class: 24%.
#### Logistic Regression:

Sex, Fare, and Pclass are key survival predictors.

In [20]:
# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.simplefilter("ignore")

In [21]:
# Fill missing values
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

# Convert categorical variables into numerical
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
df["Embarked"] = df["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# Feature Selection
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = df[features]
y = df["Survived"]

# Standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Display dataset shape
print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)

Training Data Shape: (712, 7)
Testing Data Shape: (179, 7)


##### Fixes Applied:

- Dropped unnecessary columns (Name, Ticket, Cabin).
- Filled missing values (Age, Embarked).
- Converted categorical variables (Sex, Embarked).
- Standardized numerical features (Age, Fare).


#### Train Machine Learning Models
Logistic Regression Model

In [23]:
print("Missing Values:\n", df.isnull().sum())  # Check missing values
print("\nInfinite Values:\n", np.isinf(X_scaled).sum())  # Check infinite values
print("\nVery Large Values:\n", (X_scaled > 1e6).sum())  # Check extremely large values


Missing Values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex            891
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked       891
dtype: int64

Infinite Values:
 0

Very Large Values:
 0


In [25]:
# Standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Replace infinities with large finite numbers
X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=1e6, neginf=-1e6)
